In [ ]:
!pip install pymongo

In [1]:
import pprint
import re
import requests
import tarfile
from datetime import datetime
from glob import glob
from pymongo import MongoClient
import key_param

In [4]:
url = 'https://www.rondhuit.com/download/ldcc-20140209.tar.gz'
target_path = 'ldcc-20140209.tar.gz'

response = requests.get(url, stream=True)
if response.status_code == 200:
    with open(target_path, 'wb') as f:
        f.write(response.raw.read())


file = tarfile.open('ldcc-20140209.tar.gz')
file.extractall()
file.close

<bound method TarFile.close of <tarfile.TarFile object at 0x10742b100>>

In [3]:
topic_news_list = glob('./text/topic-news/[!LICENSE]*')
data = []
for i, news_file in enumerate(topic_news_list):
    data.append({'url':'', 'datetime_of_topic': '', 'contents': ''})
    data[i]['url']=''
    data[i]['datetime_of_topic'] = ''
    data[i]['title'] = ''
    data[i]['contents'] = ''
    data[i]['relative_articles'] = []
    data[i]['relative_info'] = []
    k=0
    with open(topic_news_list[i]) as f:
        for j, line in enumerate(f):
            if j==0:
                data[i]['url']=line.strip('\n')
            elif j==1:
                data[i]['datetime_of_topic'] = datetime.strptime(line.strip('\n'), '%Y-%m-%dT%H:%M:%S%z')
            elif j==2:
                data[i]['title'] = line.strip('\n')
            else:
                if re.search("関連記事", line):
                    k=1
                elif re.search("関連情報", line):
                    k=2
                else:
                    if k == 0:
                        data[i]['contents'] = data[i]['contents'] + line
                    elif k == 1:
                        if line.strip('\n') != '':
                            data[i]['relative_articles'].append(line.strip('\n'))
                    elif k == 2:
                        if line.strip('\n') != '':
                            data[i]['relative_info'].append(line.strip('\n'))
                
                



In [7]:
data[0]

{'url': 'http://news.livedoor.com/article/detail/6601493/',
 'datetime_of_topic': datetime.datetime(2012, 5, 28, 12, 55, tzinfo=datetime.timezone(datetime.timedelta(seconds=32400))),
 'contents': '25日、母親の生活保護受給について謝罪会見を行ったお笑いコンビ「次長課長」の河本準一が、翌日ツイッター上でも改めて謝罪した。\n\nその発言を受け、お笑いコンビ「オリエンタルラジオ」の藤森慎吾が河本へ激励のコメントを送ったが、それを一般ユーザーが批判し、藤森もこれに応戦したため、ネット掲示板で大きな話題となった。\n\n発端は藤森が河本の謝罪に対して「笑わせてくれればいいです！」と激励のツイートをしたことに始まる。このコメントを見た一般ユーザーが「（河本の母親が生活保護受給していたことは）立派な詐欺なのによくこういうこと言えるね芸人って」とツイッター上で批判したが、藤森は「芸人全般に対する侮辱ですかー？謝罪はしたけど詐欺ではないでしょ。自分の発言に責任を持った方がうぃーねー！」と反論。\n\nさらに別のユーザーから「身勝手で無責任な事ほざくな。お前も社会人なら世の中に対する責任あるだろ」と指摘されると、「少なくともあなたよりは社会人としての自覚はあります。こういう公の場であなたのような乱暴で幼稚な言葉使いはしないからー」と返している。\n\n他にも、藤森は「ちょっとなにいってるかわかりましぇーーん！」「はいさようならー！ブロックー！」など、一般ユーザーに対して煽るような口調で返信したため、一連のやり取りが掲示板やツイッター上で広がると、ネットユーザーからは「藤森も黙ってりゃいいのに…なんでみんな自分からコメントしたがるのかね 」「芸人同士で擁護しあって、自分が不祥事起こしたときに擁護してもらおうという算段か」などと批判的な意見が相次いだが、中には「吉本芸人が先輩を批判できるわけないだろ」「芸人ってくくりで侮辱されたのがむかついたんだろうな」「ツイッターでもキャラ保ってるのは凄い」といった発言も見られた。\n\n',
 'title': '河本の生活保護問題で、オリラジ・藤森がツイ

In [ ]:
# js function for atlas trigger
```
exports = async function(changeEvent) {
    // Get the full document from the change event.
    const doc = changeEvent.fullDocument;

    // Define the OpenAI API url and key.
    const url = 'https://api.openai.com/v1/embeddings';
    // Use the name you gave the value of your API key in the “Values” utility inside of App Services
    const openai_key = context.values.get("openAIKey");

    try {
        console.log(`Processing document with id: ${JSON.stringify(doc)}`);
        

        // Call OpenAI API to get the embeddings.
        let response = await context.http.post({
            url: url,
             headers: {
                'Authorization': [`Bearer ${openai_key}`],
                'Content-Type': ['application/json']
            },
            body: JSON.stringify({
                input: `${doc.title} - ${doc.contents}`,
                model: "text-embedding-ada-002"
            })
        });

        // Parse the JSON response
        let responseData = EJSON.parse(response.body.text());

        // Check the response status.
        if(response.statusCode === 200) {
            console.log("Successfully received embedding.");

            const embedding = responseData.data[0].embedding;

            // Get the cluster in MongoDB Atlas.
            const mongodb = context.services.get('mongodb-atlas');
            const db = mongodb.db('livedoor_news'); // Replace with your database name.
            const collection = db.collection('topic_news'); // Replace with your collection name.

            // Update the document in MongoDB.
            const result = await collection.updateOne(
                { _id: doc._id },
                // The name of the new field you’d like to contain your embeddings.
                { $set: { embeddings: embedding }}
            );

            if(result.modifiedCount === 1) {
                console.log("Successfully updated the document.");
                
                
            } else {
                console.log("Failed to update the document.");
            }
        } else {
            console.log(`Failed to receive embedding. Status code: ${response.body.text()}`);
        }

    } catch(err) {
        console.error(err);
    }
};
```

In [6]:
client = MongoClient(key_param.MONGO_URI)
db = client['livedoor_news']
collection = db['topic_news']

if collection.count_documents({}) != 0:
    collection.delete_many({})

result = collection.insert_many(data)
print(f'{len(result.inserted_ids)} documents are inserted')

770 documents are inserted


In [ ]:
# 注：この先に進む前にAtlas UIでlangchain_demo.collection_of_text_blobsコレクションに対して
# 以下のVector Searchのインデックスを作成する

#index名: topic_news_openai

'''
{
  "fields": [{
    "path": "embeddings",
    "numDimensions": 1536,
    "similarity": "cosine",
    "type": "vector"
  }]
}
'''